### Download model



In [1]:
!wget https://storage.googleapis.com/dm-tapnet/tapnext/bootstapnext_ckpt.npz
!wget https://storage.googleapis.com/dm-tapnet/tapnext/tapnext_ckpt.npz

--2025-05-21 23:01:56--  https://storage.googleapis.com/dm-tapnet/tapnext/bootstapnext_ckpt.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.207, 142.250.98.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 776980182 (741M) [application/octet-stream]
Saving to: ‘bootstapnext_ckpt.npz.1’

bootstapnext_ckpt.n 100%[===================>] 740.99M  95.3MB/s    in 6.9s    

2025-05-21 23:02:03 (108 MB/s) - ‘bootstapnext_ckpt.npz.1’ saved [776980182/776980182]

--2025-05-21 23:02:03--  https://storage.googleapis.com/dm-tapnet/tapnext/tapnext_ckpt.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.207, 142.250.98.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 776980182 (741M) [applicatio

### Download dataset

In [2]:
!wget --no-check-certificate https://storage.googleapis.com/dm-tapnet/tapvid_davis.zip
!unzip tapvid_davis.zip

--2025-05-21 23:02:11--  https://storage.googleapis.com/dm-tapnet/tapvid_davis.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.207, 142.250.98.207, 142.251.107.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1668710491 (1.6G) [application/zip]
Saving to: ‘tapvid_davis.zip.1’

tapvid_davis.zip.1  100%[===================>]   1.55G   169MB/s    in 12s     

2025-05-21 23:02:22 (136 MB/s) - ‘tapvid_davis.zip.1’ saved [1668710491/1668710491]

Archive:  tapvid_davis.zip
replace tapvid_davis/SOURCES.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace tapvid_davis/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
import torch
import torchvision

In [2]:
torch.__version__, torchvision.__version__

('2.6.0+cu124', '0.21.0+cu124')

In [ ]:
!pip install -q git+https://github.com/google-deepmind/tapnet.git

In [ ]:
!pip install -q git+https://github.com/google-deepmind/recurrentgemma.git@main

In [ ]:
!pip install "numpy<2.1.0"

In [3]:
import tqdm

In [ ]:
!git clone https://github.com/google-deepmind/tapnet.git tapnet-code

In [4]:
cd tapnet-code

/content/tapnet-code


In [6]:
from tapnet import evaluation_datasets

davis_dataset = evaluation_datasets.create_davis_dataset(
    davis_points_path='tapvid_davis/tapvid_davis.pkl',
    query_mode='first',
    # full_resolution=False,
    resolution=(256, 256),
)

cached_dataset = []
for j, batch in enumerate(davis_dataset):
  cached_dataset.append(batch)
  print(
      'video id',
      j,
  )

30it [00:13,  2.19it/s]

video id 0
video id 1
video id 2
video id 3
video id 4
video id 5
video id 6
video id 7
video id 8
video id 9
video id 10
video id 11
video id 12
video id 13
video id 14
video id 15
video id 16
video id 17
video id 18
video id 19
video id 20
video id 21
video id 22
video id 23
video id 24
video id 25
video id 26
video id 27
video id 28
video id 29


### TAPNext

In [7]:
import numpy as np
from tapnet.tapnext.tapnext_torch import TAPNext
from tapnet.tapnext.tapnext_torch_utils import restore_model_from_jax_checkpoint, tracker_certainty
import torch.nn.functional as F

/usr/local/lib/python3.11/dist-packages/torch/_export/utils.py:415: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _register_pytree_node(


In [8]:
def run_eval_per_frame(
    model,
    batch,
    get_trackwise_metrics=True,
    radius=8,
    threshold=0.5,
    use_certainty=False,
):
  with torch.no_grad():
    pred_tracks, track_logits, visible_logits, tracking_state = model(
        video=batch['video'][:, :1], query_points=batch['query_points']
    )
    pred_visible = visible_logits > 0
    pred_tracks, pred_visible = [pred_tracks.cpu()], [pred_visible.cpu()]
    pred_track_logits, pred_visible_logits = [track_logits.cpu()], [
        visible_logits.cpu()
    ]
    for frame in tqdm.tqdm(range(1, batch['video'].shape[1])):
      # ***************************************************
      # HERE WE RUN POINT TRACKING IN PURELY ONLINE FASHION
      # ***************************************************
      (
          curr_tracks,
          curr_track_logits,
          curr_visible_logits,
          tracking_state,
      ) = model(
          video=batch['video'][:, frame : frame + 1],
          state=tracking_state,
      )
      curr_visible = curr_visible_logits > 0
      # ***************************************************
      pred_tracks.append(curr_tracks.cpu())
      pred_visible.append(curr_visible.cpu())
      pred_track_logits.append(curr_track_logits.cpu())
      pred_visible_logits.append(curr_visible_logits.cpu())
    tracks = torch.cat(pred_tracks, dim=1).transpose(1, 2)
    pred_visible = torch.cat(pred_visible, dim=1).transpose(1, 2)
    track_logits = torch.cat(pred_track_logits, dim=1).transpose(1, 2)
    visible_logits = torch.cat(pred_visible_logits, dim=1).transpose(1, 2)

    pred_certainty = tracker_certainty(tracks, track_logits, radius)
    pred_visible_and_certain = (
        F.sigmoid(visible_logits) * pred_certainty
    ) > threshold

    if use_certainty:
      occluded = ~(pred_visible_and_certain.squeeze(-1))
    else:
      occluded = ~(pred_visible.squeeze(-1))

  scalars = evaluation_datasets.compute_tapvid_metrics(
      batch['query_points'].cpu().numpy(),
      batch['occluded'].cpu().numpy(),
      batch['target_points'].cpu().numpy(),
      occluded.numpy() + 0.0,
      tracks.numpy()[..., ::-1],
      query_mode='first',
      get_trackwise_metrics=get_trackwise_metrics,
  )
  return (
      tracks.numpy()[..., ::-1],
      occluded,
      {k: v.sum(0) for k, v in scalars.items()},
  )


# @title Function for raw data to the input format {form-width: "25%"}
def deterministic_eval(cached_dataset, strided=False):
  if not strided:
    for sample in tqdm.tqdm(cached_dataset, disable=True):
      batch = sample['davis'].copy()
      # batch['video'] = (batch['video'] + 1) / 2
      batch['visible'] = np.logical_not(batch['occluded'])[..., None]
      batch['padding'] = np.ones(
          batch['query_points'].shape[:2], dtype=np.bool_
      )
      batch['loss_mask'] = np.ones(
          batch['target_points'].shape[:3] + (1,), dtype=np.float32
      )
      batch['appearance'] = np.ones(
          batch['target_points'].shape[:3] + (1,), dtype=np.float32
      )

      yield batch
  else:
    for sample in tqdm.tqdm(cached_dataset):
      batch = sample['davis'].copy()
      # batch['video'] = (batch['video'] + 1) / 2
      batch['visible'] = np.logical_not(batch['occluded'])[..., None]
      batch['padding'] = np.ones(
          batch['query_points'].shape[:2], dtype=np.bool_
      )
      batch['loss_mask'] = np.ones(
          batch['target_points'].shape[:3] + (1,), dtype=np.float32
      )
      batch['appearance'] = np.ones(
          batch['target_points'].shape[:3] + (1,), dtype=np.float32
      )
      backward_batch = {k: v.copy() for k, v in batch.items()}
      for key in ['visible', 'appearance', 'loss_mask', 'target_points']:
        backward_batch[key] = np.flip(backward_batch[key], axis=2)
      backward_batch['video'] = np.flip(backward_batch['video'], axis=1)
      backward_queries = (
          backward_batch['video'].shape[1]
          - backward_batch['query_points'][..., 0]
          - 1
      )
      backward_batch['query_points'][..., 0] = backward_queries
      yield batch, backward_batch

### Create the model and load checkpoint

In [10]:
model = TAPNext(image_size=(256, 256))
ckpt_path = '/content/bootstapnext_ckpt.npz'
model = restore_model_from_jax_checkpoint(model, ckpt_path)
model.cuda()

TAPNext(
  (lin_proj): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
  (blocks): ModuleList(
    (0-11): 12 x TRecViTBlock(
      (ssm_block): ResidualBlock(
        (temporal_pre_norm): RMSNorm()
        (recurrent_block): RecurrentBlock(
          (linear_y): Linear(in_features=768, out_features=768, bias=True)
          (linear_x): Linear(in_features=768, out_features=768, bias=True)
          (linear_out): Linear(in_features=768, out_features=768, bias=True)
          (conv_1d): CausalConv1D()
          (rg_lru): RGLRU(
            (input_gate): BlockDiagonalLinear()
            (a_gate): BlockDiagonalLinear()
          )
        )
        (channel_pre_norm): RMSNorm()
        (mlp_block): MLPBlock(
          (ffw_up): Einsum()
          (ffw_down): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (vit_block): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (

### Run inference

In [11]:
standard_eval_scalars_list = []
preds = []
for batch in deterministic_eval(cached_dataset):
  batch = {k: torch.from_numpy(v).cuda().float() for k, v in batch.items()}
  with torch.amp.autocast('cuda', dtype=torch.float16, enabled=True):
    tracks, occluded, scores = run_eval_per_frame(
        model, batch, get_trackwise_metrics=False, use_certainty=False
    )
  standard_eval_scalars_list.append(scores)
  preds.append((tracks, occluded))


print('')
print(
    'AJ',
    np.mean([
        standard_eval_scalars_list[k]['average_jaccard']
        for k in range(len(standard_eval_scalars_list))
    ]),
)
print(
    'OA',
    np.mean([
        standard_eval_scalars_list[k]['occlusion_accuracy']
        for k in range(len(standard_eval_scalars_list))
    ]),
)
print(
    'PTS',
    np.mean([
        standard_eval_scalars_list[k]['average_pts_within_thresh']
        for k in range(len(standard_eval_scalars_list))
    ]),
)

TypeError: expected np.ndarray (got int)